# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [2]:
import numpy as np
from scipy import linalg
from scipy import optimize
import sympy as sm

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

# Model description

Consider the **standard Solow-model** where:

1. $K_t$ is capital
2. $H_t$ is total human capital ($H_t = h_t * L_t$)
3. $L_t$ is labor (growing with a constant rate of $n$)
4. $A_t$ is technology (growing with a constant rate of $g$)
5. $Y_t = F(K_t,A_tL_t)$ is GDP
6. $s_K$ is the saving rate of capital
7. $s_H$ is the saving rate of human capital

**Saving** is a constant fraction of GDP

$$ 
S_t = (s_{K} + s_{H})Y_t,\,s_{K},s_{H}\in(0,1)
$$

such that **capital accumulates** according to

$$
K_{t+1}=s_{K}Y_{t}+(1-\delta)K_{t}=s_{K}F(K_{t}H_{t},A_{t}L_{t})+(1-\delta)K_{t}, \delta \in (0,1)
$$

and that **human capital accumulates** according to

$$
H_{t+1}=s_{H}Y_{t}+(1-\delta)H_{t}=s_{H}F(K_{t}H_{t},A_{t}L_{t})+(1-\delta)H_{t}, \delta \in (0,1)
$$

The **production function** has **constant-return to scale** such that

$$
\frac{Y_{t}}{A_{t}L_{t}}=\frac{F(K_{t}H_{t},A_{t}L_{t})}{A_{t}L_{t}}=F(\tilde{k}_{t},\tilde{h}_{t},1)\equiv f(\tilde{k}_{t},\tilde{h}_{t})
$$

where $\tilde{k}_t = \frac{K_t}{A_{t}L_{t}}$ is the technology adjusted capital-labor ratio and $\tilde{h}_t = \frac{H_t}{A_{t}L_{t}}$ is the technology adjusted human capital-labor ratio

The **transition equations** then becomes

$$
\tilde{k}_{t+1}= \frac{1}{(1+n)(1+g)}[s_{K}f(\tilde{k}_{t},\tilde{h}_{t})+(1-\delta)\tilde{k}_{t}]
$$

$$
\tilde{h}_{t+1}= \frac{1}{(1+n)(1+g)}[s_{H}f(\tilde{k}_{t},\tilde{h}_{t})+(1-\delta)\tilde{h}_{t}]
$$

If the **production function** is **Cobb-Douglas** then

$$
F(K_{t}H_{t},A_{t}L_{t})=K_{t}^{\alpha}H_{t}^{\phi}(A_{t}L_{t})^{1-\alpha-\phi}\Rightarrow f(\tilde{k}_{t},\tilde{h}_{t})=\tilde{k}_{t}^{\alpha}\tilde{h}_{t}^{\phi}
$$



## Analytical solution

To find the steady states of the two production factors we set $k_t = k_{t+1}$ and $h_t=h_{t+1}$

$$
\tilde{k}^{\ast}= \frac{1}{(1+n)(1+g)}[s_{K}f(\tilde{k}^{\ast},\tilde{h}^{\ast})+(1-\delta)\tilde{k}^{\ast}]
$$

$$
\tilde{h}^{\ast}= \frac{1}{(1+n)(1+g)}[s_{H}f(\tilde{k}^{\ast},\tilde{h}^{\ast})+(1-\delta)\tilde{h}^{\ast}]
$$

In [28]:
k = sm.symbols('k')
h = sm.symbols('h')
alpha = sm.symbols('alpha')
delta = sm.symbols('delta')
phi = sm.symbols('phi')
sk = sm.symbols('s_K')
sh = sm.symbols('s_H')
g = sm.symbols('g')
n = sm.symbols('n')

In [36]:
# Define f(k)
f = k**alpha*h**phi

# Transition equation for k and h to find SS
ssk = sm.Eq(0,(sk*f-(n+g+delta+n*g)*k)/((1+n)*(1+g)))
ssh = sm.Eq(0,(sh*f-(n+g+delta+n*g)*h)/((1+n)*(1+g)))

# Isolate for k and h in transition eq.
k1 = sm.solve(ssk,k)[0]

# Display solution
ssh

Eq(0, (-h*(delta + g*n + g + n) + h**phi*k**alpha*s_H)/((g + 1)*(n + 1)))

In [37]:
k1

((delta + g*n + g + n)/(h**phi*s_K))**(1/(alpha - 1))

In [38]:
ssh_sub = ssh.subs(k,k1)

ssh_sub


Eq(0, (-h*(delta + g*n + g + n) + h**phi*s_H*(((delta + g*n + g + n)/(h**phi*s_K))**(1/(alpha - 1)))**alpha)/((g + 1)*(n + 1)))

In [39]:
hss = sm.solve(ssh_sub,h)

hss

NotImplementedError: multiple generators [h, ((delta/(h**phi*s_K) + g*n/(h**phi*s_K) + g/(h**phi*s_K) + n/(h**phi*s_K))**(1/(alpha - 1)))**alpha, h**phi]
No algorithms are implemented to solve equation -h*(delta + g*n + g + n) + h**phi*s_H*(((delta + g*n + g + n)/(h**phi*s_K))**(1/(alpha - 1)))**alpha

In [ ]:
# Use sm.lambdify to turn the solution into 
ss_func = sm.lambdify(args=(s,g,n,delta,alpha),expr=kss)

# Evaluate the function
ss_func(0.2,0.02,0.01,0.1,1/3)

1.903831539231319

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

In [ ]:
s = 0.2
g = 0.02
n = 0.01
alpha = 1/3
delta = 0.1

# Define f
f = lambda k: k**alpha

# Define the objective function
obj_kss = lambda kss: kss - (s*f(kss) + (1-delta)*kss)/((1+g)*(1+n))

# Find result
result = optimize.root_scalar(obj_kss,bracket=[0.1,100],method='brentq')

# Print result
print('the steady state for k is',result.root)   

the steady state for k is 1.9038315392313185


### CES

In [ ]:
betas = [-0.5,-0.25,-0.1,-0.05,0.05,0.1,0.25,0.5]

for beta in betas:
    # Define f
    f = lambda k: (alpha*k**beta + (1-alpha))**(1/beta)

    # Define the objective function
    obj_kss = lambda kss: kss - (s*f(kss) + (1-delta)*kss)/((1+g)*(1+n))

    # Find result
    result = optimize.root_scalar(obj_kss,bracket=[0.1,100],method='brentq')

    # Print result
    print(f'for beta = {beta:.3f} the steady state for k is',result.root) 

for beta = -0.500 the steady state for k is 1.8471297000972984
for beta = -0.250 the steady state for k is 1.873383262758588
for beta = -0.100 the steady state for k is 1.8910856397655083
for beta = -0.050 the steady state for k is 1.8973581025712734
for beta = 0.050 the steady state for k is 1.9105159729244352
for beta = 0.100 the steady state for k is 1.917422132817728
for beta = 0.250 the steady state for k is 1.9395902733676993
for beta = 0.500 the steady state for k is 1.9822334997701418


# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 